In [ ]:
using DataFrames
using ControlSystemsBase
using Statistics
using Distributions
using Plots
using JLD
using GLM

using RealTimeScheduling
using ControlTimingSafety

push!(LOAD_PATH, "../lib")
using Experiments
using Benchmarks

## SHT

Address the issue about equal prior belief for $H_0$ and $H_1$. Recall that given deviation upper bound $d_{ub}$, confidence level $c$, and the set of all possible samples $X$, the two hypothesis are defined as follows:
$$
\DeclareMathOperator{\dev}{Deviation}
H_0: Pr(\dev(x) < d_{ub} | x \in X) < c \\
H_1: Pr(\dev(x) < d_{ub} | x \in X) > c
$$
assuming that the sample $x$ is drawn uniformly from the set $X$.



In the paper, the Bayes Factor $B$ is used to compute $K$, the number of required samples to draw to give 
\begin{align}
\frac{Pr(H_1)}{Pr(H_0)} B &= \frac{Pr(H_1|Data)}{Pr(H_0|Data)} \\
\frac{Pr(H_1)}{Pr(H_0)} B &= \frac{\frac{Pr(Data|H_1)Pr(H_1)}{Pr(Data)}}{\frac{Pr(Data|H_0)Pr(H_0)}{Pr(Data)}} \\
B &= \frac{Pr(Data|H_1)}{Pr(Data|H_0)} \\
B &= \frac{\int_c^1 P(Data|\theta=q, H_1) f_{\theta|H_1}(q)dq}
          {\int_0^c P(Data|\theta=q, H_0) f_{\theta|H_0}(q)dq} \\
B &= \frac{\int_c^1 P(Data|\theta=q, H_1) \frac{1}{1-c} dq}
          {\int_0^c P(Data|\theta=q, H_0) \frac{1}{c} dq} \\
B &= \frac{c \int_c^1 q^K dq}
          {(1-c) \int_0^c q^K dq} \\
\frac{1-c}{c} B &= \frac{\int_c^1 q^K dq}
                        {\int_0^c q^K dq} \\
K &= -\log_c(\frac{1-c}{c}B + 1)
\end{align}

<!-- Traditionally, a Bayes factor $B$ of $100$ is considered extreme evidence favoring the alternative hypothesis. However, the paper we are based on states the type-I error is expressed by
$$
err = \frac{c}{c(1-c)B}
$$
Which is ~$.497$ when $B=100$, and ~$.000238$ when $B=4.15e5$. -->

In [ ]:
let
    c = 0.99
    B = 100
    e = c/(c+(1-c)*B)
    B_og = 4.15e5
    e_og = c/(c+(1-c)*B_og)
    [e e_og]
end

In [ ]:
let
    c = [0.9, 0.99, 0.999, 0.9999, 1-1e-10, 1-1e-15, 1-1e-16]
    [-log.(c, c./ (1 .- c)) -log.(c, 4.15e5+1)]
end

In [ ]:
let
    B = 4.15e5
    c = [0.9, 0.99, 0.999, 0.9999, 1-1e-10, 1-1e-15, 1-1e-16]
    # effective_B = c ./ (1 .- c) * B
    effective_B = (1 .- c) ./ c * B
    K = -log.(c, effective_B .+ 1)
    K_og = -log.(c, B+1)

    round.([c K_og K K_og./K], sigdigits=4)
end

In [ ]:
let
    B = 100
    c = [0.9, 0.99, 0.999, 0.9999, 1-1e-10, 1-1e-15, 1-1e-16]
    # effective_B = c ./ (1 .- c) * B
    effective_B = (1 .- c) ./ c * B
    K = -log.(c, effective_B .+ 1)
    K_og = -log.(c, B+1)

    round.([c K_og K], sigdigits=4)
end

## Experiments

### pWCET Sampler

In [ ]:
sp = SamplerPWCET(0.9, 100)

In [ ]:
mean([sum(rand(sp)) for i in 1:100])

### System Dynamics

Open-loop stable system:
$$
\mathbf{\dot{x}}(t) = \begin{bmatrix} -6.0 & 1.0 \\ 0.2 & -0.7 \end{bmatrix} \mathbf{x}(t) +
\begin{bmatrix} 5.0 \\ 0.5 \end{bmatrix} u(t)
$$
where
$$
\mathbf{x}(t) = \begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix}
$$
is the system state and $u(t)$ is the control input.

We also have a open-loop unstable system:
$$
\mathbf{\dot{x}}(t) = \begin{bmatrix} 0 & 6.5 \\ 0 & 0 \end{bmatrix} \mathbf{x}(t) +
\begin{bmatrix} 0 \\ 19.685 \end{bmatrix} u(t)
$$

In [ ]:
benchmarks[:RCN], benchmarks[:F1T]

In [ ]:
# Setting parameters for the experiment
H = 100
c = 0.99
B = 4.15e5
h = 0.02
sys = benchmarks[:F1T]
x0 = 1.
u0 = 0.
z0 = [fill(x0, size(sys.A, 1)); u0]

# Construct an automaton with no constraint
a = hold_kill(c2d(sys, h), delay_lqr(sys, h))

### Simulations

In [ ]:
n = 100000
res99 = sim(a, z0, 0.99, n)
res90 = sim(a, z0, 0.90, n)
res50 = sim(a, z0, 0.50, n)

In [ ]:
n = 100000
res90 = sim(a, z0, 0.90, n)

In [ ]:
res10 = sim(a, z0, 0.10, n)

In [ ]:
save("../data/res99.jld", "res99", res99)
save("../data/res99.jld", "res90", res90)
save("../data/res50.jld", "res50", res50)

In [ ]:
save("../data/res50.jld", "res10", res10)

In [ ]:
devs99 = map(x -> x[2], res99)
sigm99 = map(x -> x[1], res99);

In [ ]:
map(filter(x -> misstotal(x[1]) < 3, res99)) do x
    (missfirst(x[1]), x[2])
end

In [ ]:
map(collect(filter(x -> misstotal(x[1]) < 3, res99))[end-3:end]) do x
    println(x[1], x[2])
end

In [ ]:
res = res90
devs = map(x -> x[2], res)
σs = map(x -> x[1], res)
σs3 = map(x -> x[1:3], σs)
σs5 = map(x -> x[1:5], σs)
σs7 = map(x -> x[1:7], σs)
σsx = map(x -> x[1:10], σs)
σx = σs7
p1 = plot(devs, (1:length(res)) ./ length(res), 
    xlabel="Deviation", ylabel="Probability", label="cdf", title="p=0.10")
p2 = plot(missrow.(σx), devs,
    ylabel="Deviation", xlabel="Consecutive Misses", label="p=0.9")
p3 = plot(misstotal.(σx), devs,
    ylabel="Deviation", xlabel="Total Misses")
p4 = plot(missfirst.(σs), devs,
    ylabel="Deviation", xlabel="First Miss")
for c in [0.99]
    dev = devs[round(Int64, length(res)*c)]
    scatter!(p1, [dev], [c], label="99th quantile = $(round(dev, sigdigits=3))")
end
println(length(res))
savefig(p2, "cv_cm_7.pdf")
savefig(p3, "cv_tm_7.pdf")
display(p3)

In [ ]:
for res in [res99, res90, res50, res10]
    devs = map(x -> x[2], res)
    dev = devs[round(Int64, length(res)*0.99)]
    println(round.([dev, devs[end], dev/devs[end]], sigdigits=3))
end

### Deviation Estimation

In [ ]:
for p in [0.1, 0.5, 0.9, 0.99]
    mind = Inf
    maxd = 0.
    sample_t = 0
    devest_t = 0
    for i in 1:50
        t = time()
        sp = SamplerPWCET(p, 100)
        sample_t += time() - t

        t = time()
        dub = estimate_deviation(a, sp, z0, c, B)
        devest_t += time() - t

        mind = min(mind, dub)
        maxd = max(maxd, dub)
    end
    @info p sample_t devest_t mind maxd
end

In [ ]:
for p in [0.1, 0.5, 0.9, 0.99]
    mind = Inf
    maxd = 0.
    sample_t = 0
    devest_t = 0
    for i in 1:50
        t = time()
        sp = SamplerPWCET(p, 100)
        sample_t += time() - t

        t = time()
        dub = estimate_deviation(a, sp, z0, c, B)
        devest_t += time() - t

        mind = min(mind, dub)
        maxd = max(maxd, dub)
    end
    @info p sample_t devest_t mind maxd maxd/mind
end

In [ ]:
mind = Inf
maxd = 0.
x0 = 1.
u0 = 0.
for i in 1:50
    sp = SamplerPWCET(0.4, 100)
    # Construct an automaton with no constraint
    a = hold_kill(c2d(sys, h), delay_lqr(sys, h))
    dub = estimate_deviation(a, sp, [fill(x0, size(sys.A, 1)); u0], c, B)
    mind = min(mind, dub)
    maxd = max(maxd, dub)
end
mind, maxd

In [ ]:
mind = Inf
maxd = 0.
sys = benchmarks[:F1T]
x0 = 1.
u0 = 0.
for i in 1:50
    sp = SamplerPWCET(0.9, 100)
    # Construct an automaton with no constraint
    a = hold_kill(c2d(sys, h), delay_lqr(sys, h))
    dub = estimate_deviation(a, sp, [fill(x0, size(sys.A, 1)); u0], c, B)
    mind = min(mind, dub)
    maxd = max(maxd, dub)
end
mind, maxd

In [ ]:
mind = Inf
maxd = 0.
sys = benchmarks[:F1T]
x0 = 1.
u0 = 0.
for i in 1:50
    sp = SamplerPWCET(0.99, 100)
    # Construct an automaton with no constraint
    a = hold_kill(c2d(sys, h), delay_lqr(sys, h))
    dub = estimate_deviation(a, sp, [fill(x0, size(sys.A, 1)); u0], c, B)
    mind = min(mind, dub)
    maxd = max(maxd, dub)
end
mind, maxd

### Likelihood Ratio Test

\begin{align}
\lambda_{LR} &=\frac{\sup_{\theta_0\in\Theta_0}{L}(\theta_0)}{\sup_{\theta\in\Theta}L(\theta)} \\
&= \frac{\binom{n}{x} \theta_0^x (1-\theta_0)^{n-x}}{\binom{n}{x} \theta^x (1-\theta)^{n-x}} \\
&= \frac{\theta_0^x (1-\theta_0)^{n-x}}{ \theta^x (1-\theta)^{n-x}} \\
&= \left(\frac{\theta_0}{\theta}\right)^x + \left(\frac{1-\theta_0}{1-\theta}\right)^{n-x}
\end{align}

\begin{align}
&= -2 \ln \left[ \frac{\binom{n}{k} \theta_0^x (1-\theta_0)^{n-x}}{\binom{n}{k} \theta_1^x (1-\theta_1)^{n-x}} \right] \\
&= -2 \ln \left[ \frac{\theta_0^x (1-\theta_0)^{n-x}}{ \theta_1^x (1-\theta_1)^{n-x}} \right] \\
&= -2 (x \ln \frac{\theta_0}{\theta_1} + (n-x) \ln \frac{1-\theta_0}{1-\theta_1})
\end{align}

In [ ]:
lr_test(0.99, 100, 99)

In [ ]:
lr_test(0.99, 830, 830)

In [ ]:
lr_test_2(0.99, 8782, 0.005)

In [ ]:
lr_test_2(0.99, 46500, 0.002)

Likelihood ratio with null hypothesis: true $\theta$ is within a range (e.g., $[0.899, 0.991]$) instead of $\theta>0.99$ vs $\theta<0.99$.

In [ ]:
err = 0.000238497

In [ ]:
n = 8782
for p in [0.99, 0.90, 0.50, 0.10]
    mind = Inf
    maxd = 0.
    devest_t = 0
    for i in 1:50
        t = time()
        lrres = sim(σ p, n)
        x = round(Int64, n * 0.99)
        dev = lrres[x][2]
        devest_t += time() - t
        
        mind = min(mind, dev)
        maxd = max(maxd, dev)
    end
    @info p devest_t mind maxd maxd/mind
end

### Control Variate

In [ ]:
# Using values from simulation

res = res90
devs = map(x -> x[2], res)
σs = map(x -> x[1], res)
σs5 = map(x -> x[1:10], σs)

X = devs
Y = missrow.(σs)
Y5 = missrow.(σs5)
Z = misstotal.(σs)
Z5 = misstotal.(σs5)
U = missfirst.(σs)

data = DataFrame(Dev=X, CM=Y, TM=Z, FM=U, CM5=Y5, TM5=Z5)

ols_y = lm(@formula(Dev ~ CM5), data)

In [ ]:
ols_z = lm(@formula(Dev ~ TM5), data)

In [ ]:
ols_u = lm(@formula(Dev ~ FM), data)

In [ ]:
FcvW(0.1566, 10, res90, misstotal)

In [ ]:
inverse_fcv(0.99, x, res50, missfirst)

In [ ]:
calculate_mean_miss(misstotal,res99)

## Analytic solution

### Mean for first miss
The probability $P(n)$ of first miss occurs in position $n$ in a string of length $k (k \ge n)$ with hit probablity $a (0\le a\le1)$ is:
$$
P(n)=a^{n-1}\cdot (1-a)
$$
The mean value of the position of first miss $M(k)$ can be expressed as:
$$
\begin{align}
M(k)&=\sum^{k}_{n=1}nP(n)\\
&=(1-a)\sum^{k}_{n=1}na^{n-1}\\
&=(1-a)\sum^{k}_{n=1}\frac{d}{da}a^n\\
&=(1-a)\frac{d}{da}\sum^{k}_{n=1}a^n\\
&=(1-a)\frac{d}{da}\frac{a(1-a^k)}{1-a}\\
&=\frac{ka^{k+1}-(k+1)a^k+1}{1-a}
\end{align}
$$

### Mean for total miss
The probability $P(n)$ of $n$ total misses occurs  in a string of length $k (k \ge n)$ with hit probablity $a (0\le a\le1)$ is:
$$
P(n)= {k \choose n}a^{k-n}(1-a)^n
$$
Let $1-a=x$.
The mean value of the total number of misses $M(k)$ can be expressed as:
$$
\begin{align}
M(k)&=\sum^{k}_{n=1}nP(n)\\
&=\sum^{k}_{n=1}n{k \choose n}a^{k-n}x^n\\
&=x\sum^{k}_{n=1}\frac{d}{dx}{k \choose n}a^{k-n}x^n\\
&=x\frac{d}{da}(a+x)^k\\
&=x\cdot k\\
&=(1-a)\cdot k
\end{align}